In [1]:
import logging
import pickle
import joblib
import time

import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [2]:
training_examples = pd.read_pickle("data/airlines_training_examples.pkl")
training_examples.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,CRSElapsedTime,Distance,Cancelled,...,Origin__STX,Origin__SWF,Origin__SYR,Origin__TLH,Origin__TPA,Origin__TRI,Origin__TUL,Origin__TUS,Origin__TYS,Origin__UCA
17236,1995,1,25,3,1710,1807,758,57.0,198.0,0,...,0,0,0,0,0,0,0,0,0,0
32157,2003,1,16,4,1445,1610,1027,85.0,334.0,0,...,0,0,0,0,0,0,0,0,0,0
3976,1988,1,18,1,1414,1510,1478,56.0,258.0,0,...,0,0,0,0,0,0,0,0,0,0
18567,1996,1,22,1,925,1140,420,75.0,369.0,0,...,0,0,0,0,0,0,0,0,0,0
37383,2005,1,19,3,740,855,589,135.0,642.0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test_examples = pd.read_pickle("data/airlines_test_examples.pkl")
test_examples.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,CRSElapsedTime,Distance,Cancelled,...,Origin__STX,Origin__SWF,Origin__SYR,Origin__TLH,Origin__TPA,Origin__TRI,Origin__TUL,Origin__TUS,Origin__TYS,Origin__UCA
16911,1995,1,2,1,1225,1332,2181,67.0,238.0,0,...,0,0,0,0,0,0,0,0,0,0
2393,1988,1,7,4,2059,2229,951,90.0,461.0,1,...,0,0,0,0,0,0,0,0,0,0
29869,2001,1,2,2,1921,2100,442,99.0,526.0,0,...,0,0,0,0,0,0,0,0,0,0
2493,1988,1,30,6,1850,1955,954,65.0,278.0,0,...,0,0,0,0,0,0,0,0,0,0
7501,1990,1,24,3,935,1109,53,274.0,1814.0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
f = open("data/airlines_training_targets.pkl", "rb")
training_targets = pickle.load(f)
f.close()
print(f"Shape of training examples{training_examples.shape}")
print(f"Shape of training target{training_targets.shape}")

Shape of training examples(30784, 286)
Shape of training target(30784,)


In [14]:
#Functions to get a data
def get_train_points():
    return np.asarray(training_examples.values.tolist())

def get_test_points(start_row_index, end_row_index):
    return test_examples.iloc[start_row_index:end_row_index].values.tolist()

def get_test_point(row_index):
    return test_examples.iloc[row_index].tolist()

#Create a training matrix
dtrain = xgb.DMatrix(get_train_points(),
                     label=training_targets)
                    
param = {'max_depth': 2,
         'eta': 1,
         'silent': 1,
         'objective': 'binary:logistic'}
watchlist = [(dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)

[0]	train-error:0.37926
[1]	train-error:0.37487
